In [ ]:
import logging
import pyodbc
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


def crawl_single_page(driver, page, existing_links):
    """Cào dữ liệu từ một trang cụ thể."""
    page_details = []

    try:
        logging.info(f"Đang cào dữ liệu trang {page}...")
        url = f"https://www.chotot.com/mua-ban-dien-thoai-tp-ho-chi-minh?page={page}"
        driver.get(url)

        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//div[contains(@class,"ListAds_ListAds__rEu_9")]')))

        elements = driver.find_elements(By.XPATH, '//div[contains(@class,"ListAds_ListAds__rEu_9")]//a[@itemprop="item"]')
        links = []

        for element in elements:
            link = element.get_attribute('href')
            if link and link not in existing_links:
                existing_links.add(link)
                links.append(link)

        # Cào dữ liệu cho từng sản phẩm
        for link in links:
            driver.get(link)
            product_details = [None] * 11  # Tạo danh sách chứa thông tin chi tiết sản phẩm

            try:
                elem_name = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".r9vw5if h1")))
                product_details[0] = elem_name.text

                elem_price = driver.find_elements(By.CSS_SELECTOR, 'b.p1mdjmwc')
                product_details[1] = elem_price[0].text if elem_price else None

                # Lấy chi tiết sản phẩm
                details = driver.find_elements(By.XPATH, '//div[@class="p74axq8"]//div[@class="p1vpox21"]')
                for i in range(min(5, len(details))):  # Chỉ lặp lại tối đa 5 phần tử
                    product_details[i + 2] = details[i].text  # Bắt đầu lưu trữ từ chỉ số 2

                elem_store_name = driver.find_elements(By.XPATH, '//span[@class="bfe6oav ta18pxs ti6dhdt"]')
                product_details[7] = elem_store_name[0].text if elem_store_name else None

                elem_rating = driver.find_elements(By.XPATH, '//span[@class="bfe6oav t13h538e"]')
                product_details[8] = elem_rating[0].text if elem_rating else None

                elem_review_count = driver.find_elements(By.XPATH, '//span[@class="brnpcl3 t3r6n7t"]')
                product_details[9] = elem_review_count[0].text if elem_review_count else None

                product_details[10] = link
                page_details.append(product_details)

            except (NoSuchElementException, TimeoutException) as e:
                logging.info(f"Không tìm thấy thông tin chi tiết cho sản phẩm: {link}, lỗi: {e}")
                continue

    except Exception as e:
        logging.error(f"Lỗi khi cào dữ liệu trang {page}: {e}")

    return page_details


def save_to_database(all_details):
    """Lưu danh sách chi tiết sản phẩm vào cơ sở dữ liệu Azure SQL."""
    # Kết nối cơ sở dữ liệu
    server = 'SERVER'
    database = 'DATABASE'
    username = 'USERNAME'
    password = 'PASSWORD'
    driver_sql = 'DRIVER'

    conn_str = f'DRIVER={driver_sql};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    
    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()

        # Tạo bảng nếu chưa tồn tại
        create_table_query = """
        IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'dbo.datadienthoai') AND type in (N'U'))
        BEGIN
            CREATE TABLE datadienthoai (
                Id INT PRIMARY KEY IDENTITY(1,1),
                TenSanPham NVARCHAR(255),
                Gia NVARCHAR(50),
                Hang NVARCHAR(255),
                DongMay NVARCHAR(255),
                TinhTrang NVARCHAR(255),
                ChinhSachBaoHanh NVARCHAR(255),
                MauSac NVARCHAR(255),
                TenCuaHang NVARCHAR(255),
                DiemDanhGia NVARCHAR(50),
                SoLuotDanhGia NVARCHAR(255),
                Link NVARCHAR(255)
            );
        END;
        """
        cursor.execute(create_table_query)
        conn.commit()

        insert_query = """
        INSERT INTO datadienthoai (TenSanPham, Gia, Hang, DongMay, TinhTrang, ChinhSachBaoHanh, MauSac, TenCuaHang, DiemDanhGia, SoLuotDanhGia, Link) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        for details in all_details:
            try:
                cursor.execute(insert_query, *details)
            except Exception as e:
                logging.error(f"Lỗi khi chèn dữ liệu vào cơ sở dữ liệu: {e}")

        conn.commit()
        logging.info("Dữ liệu đã được tải lên Azure SQL Database thành công.")

    except Exception as e:
        logging.error(f"Lỗi kết nối đến Azure SQL Database: {e}")

    finally:
        cursor.close()
        conn.close()


def crawl_chotot_data():
    """Chạy quá trình cào dữ liệu từ Chợ Tốt và lưu trữ vào cơ sở dữ liệu."""
    logging.basicConfig(level=logging.INFO)

    all_details = []
    existing_links = set()

    options = Options()
    options.headless = True  
    options.add_argument('--no-sandbox') 
    options.add_argument('--disable-dev-shm-usage')
    driver_service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=driver_service, options=options)

    try:
        for page in range(1, 701): 
            page_details = crawl_single_page(driver, page, existing_links)
            all_details.extend(page_details)

    finally:
        driver.quit() 

    logging.info(f"Đã cào tổng cộng {len(all_details)} sản phẩm.")

    
    save_to_database(all_details)


if __name__ == "__main__":
    crawl_chotot_data()

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [C:\Users\Surface\.wdm\drivers\chromedriver\win64\129.0.6668.91\chromedriver-win32/chromedriver.exe] found in cache
INFO:root:Đang cào dữ liệu trang 1...
INFO:root:Đang cào dữ liệu trang 2...
INFO:root:Đang cào dữ liệu trang 3...
INFO:root:Đang cào dữ liệu trang 4...
INFO:root:Đang cào dữ liệu trang 5...
INFO:root:Đang cào dữ liệu trang 6...
INFO:root:Đang cào dữ liệu trang 7...
INFO:root:Đang cào dữ liệu trang 8...
INFO:root:Đang cào dữ liệu trang 9...
INFO:root:Đang cào dữ liệu trang 10...
INFO:root:Đang cào dữ liệu trang 11...
INFO:root:Đang cào dữ liệu trang 12...
INFO:root:Đang cào dữ liệu trang 13...
INFO:root:Đang cào dữ liệu trang 14...
INFO:root:Đang cào dữ liệu trang 15...
INFO:root:Đang cào dữ liệu trang 16...
INFO:root:Đang cào dữ liệu trang 17...
INFO:root:Đang cào dữ liệu trang 18...
INFO:root:Đang c